<a href="https://colab.research.google.com/github/rajatha94/DBMSProject/blob/master/Copy_of_PS1_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this experiment, you will explore the accuracy of sentiment classificaiton using different feature representations of text documents.

First, you will implement `createBasicFeatures`, which creates a sparse matrix representation of a collection of documents. For this exercise, you should have a feature for each word containing at least one alphabetic character. You may use the `numpy` and `sklearn` packages to help with implementing a sparse matrix.

Then, you will implement `createFancyFeatures`, which can specify at any other features you choose to help improve performance on the classification task.

The two code blocks at the end train and evaluate two models—logistic regression with L1 and L2 regularization—using your featurization functions. Besides held-out classification accuracy with 10-fold cross-validation, you will also see the features in each class given high weights by the model.

In [0]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
# read in the movie review corpus
def readReviews():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/cornell_reviews.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  return corpus

This is where you will implement two functions to featurize the data.

In [0]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct class for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  #Your code here
  corpusText = [row["text"] for row in corpus]
  classes = [row["class"] for row in corpus]
  # vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
  # X_train = vectorizer.fit_transform(corpusText)
  vectorizer = CountVectorizer()
  texts = vectorizer.fit_transform(corpusText)
  vocab = vectorizer.get_feature_names()
  return texts,classes,vocab

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createFancyFeatures(corpus):
  #Your code here
  corpusText = [row["text"] for row in corpus]
  classes = [row["class"] for row in corpus]
  bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
  # X_train = vectorizer.fit_transform(corpusText)
  # vectorizer = CountVectorizer()
  texts = bigram_vectorizer.fit_transform(corpusText)
  vocab = bigram_vectorizer.get_feature_names()
  return texts,classes,vocab

In [0]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class

def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=KFold(n_splits=10, shuffle=True, random_state=1))
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  class0_weight_sorted = model.coef_[0, :].argsort()
  class1_weight_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  class0_indicators = [vocab[i] for i in class0_weight_sorted[:termsToTake]]
  class1_indicators = [vocab[i] for i in class1_weight_sorted[:termsToTake]]

  if model.classes_[0] == "pos":
    return avg_score,class0_indicators,class1_indicators
  else:
    return avg_score,class1_indicators,class0_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)

In [0]:
corpus = readReviews()
print(corpus[0])

{'class': 'neg', 'text': 'plot : two teen couples go to a church party , drink and then drive . they get into an accident . one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . what\'s the deal ? watch the movie and " sorta " find out . . . critique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . which is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn\'t snag this one correctly . they seem to have taken this pretty neat concept , but executed it terribly . so what are the problems with the movie ? well , its main problem is that it\'s simply too jumbled . it starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience 

Run the following to train and evaluate two models using basic features:

In [0]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.828500
The most informative terms for pos are: ['flaws', 'memorable', 'terrific', 'edge', 'masterpiece', 'excellent', 'perfectly', 'sherri', 'gas', 'using', 'enjoyable', 'overall', 'fun', 'follows', 'different', 'quite', 'solid', 'allows', 'fantastic', 'side']
The most informative terms for neg are: ['waste', 'mess', 'lame', 'worst', 'headed', 'ridiculous', 'unfortunately', 'awful', 'cheap', 'write', 'boring', 'tedious', 'superior', 'jesse', 'terrible', 'bad', 'poor', 'iii', 'designed', 'looks']
----------L2 Norm-----------
The model's average accuracy is 0.833000
The most informative terms for pos are: ['fun', 'great', 'back', 'quite', 'well', 'seen', 'excellent', 'perfectly', 'memorable', 'overall', 'job', 'yet', 'american', 'terrific', 'pulp', 'bit', 'true', 'performances', 'husband', 'masterpiece']
The most informative terms for neg are: ['bad', 'unfortunately', 'worst', 'waste', 'nothing', 'script', 'awful', 'boring', 

Run the following to train and evaluate two models using extended features:

In [0]:
X,y,vocab = createFancyFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.825500
The most informative terms for pos are: ['even if', 'flaws', 'memorable', 'due to', 'masterpiece', 'overall', 'follows', 'her husband', 'terrific', 'by a', 'the more', 'as much', 'works', 'seen', 'great', 'gas', 'filmmaking', 'they were', 'loved', 'images']
The most informative terms for neg are: ['ridiculous', 'waste', 'unfortunately', 'mess', 'awful', 'boring', 'worst', 'cheap', 'should have', 'lame', 'headed', 'bad', 'poor', 'write', 'maybe', '0', 'designed', 'tedious', 'terrible', 'metro']
----------L2 Norm-----------
The model's average accuracy is 0.853500
The most informative terms for pos are: ['great', 'well', 'fun', 'seen', 'very', 'back', 'quite', 'also', 'many', 'yet', 'people', 'american', 'life', 'see', 'excellent', 'while', 'job', 'the two', 'by a', 'perfectly']
The most informative terms for neg are: ['bad', 'only', 'unfortunately', 'worst', 'plot', 'any', 'nothing', 'boring', 'waste', 'script', 'poor